# Interpreter Activity 1

Find the bugs in this code!

In [1]:
data Val = IntVal Integer
         | BoolVal Bool
   deriving (Show,Eq)

In [2]:
data Exp = IntExp Integer
         | BoolExp Bool
         | IntOpExp String Exp Exp
         | BoolOpExp String Exp Exp
         | IfExp Exp Exp Exp
         | VarExp String
         | LetExp String Exp Exp
   deriving (Show,Eq)

In [3]:
type Env = [(String,Val)]

In [4]:
intOps = [ ("+",(+))
         , ("-",(-))
         , ("*",(*))
         , ("/",div)]
         
:t intOps

intOps :: forall a. Integral a => [([Char], a -> a -> a)]

In [5]:
compOps = [ (">",(>))
          , ("==",(==))]
         
:t compOps

compOps :: forall a. Ord a => [([Char], a -> a -> Bool)]

In [6]:
liftIntOp f (IntVal i1) (IntVal i2) = IntVal (f i1 i2)
liftIntOp f _           _           = IntVal 0
:t liftIntOp

liftIntOp :: (Integer -> Integer -> Integer) -> Val -> Val -> Val

In [7]:
liftCompOp f (IntVal i1) (IntVal i2) = BoolVal (f i1 i2)
liftCompOp f _           _           = BoolVal False
:t liftCompOp

liftCompOp :: (Integer -> Integer -> Bool) -> Val -> Val -> Val

In [8]:
trumpError s = error $ "You idiot! " ++ s ++ " Sad!"

In [17]:
eval :: Exp -> Env -> Val
eval (IntExp i)          _   = IntVal i
eval (BoolExp b)         _   = BoolVal b
eval (IntOpExp op e1 e2) env =
   let v1 = eval e1 env
       v2 = eval e2 env                   
    in case lookup op intOps of
         Just f -> liftIntOp f v1 v2
         Nothing -> trumpError $ op ++ " is not defined."
eval (BoolOpExp op e1 e2) env =
   let v1 = eval e1 env
       v2 = eval e2 env                   
    in case lookup op compOps of
         Just f -> liftCompOp f v1 v2
         Nothing -> trumpError $ op ++ " is not defined."
eval (IfExp e1 e2 e3) env =
   case eval e1 env of
     BoolVal True -> eval e2 env
     BoolVal False -> eval e3 env
     _             -> trumpError "Your conditional wasn't a boolean!"
eval (VarExp v) env =
   case lookup v env of
     Just v1 -> v1
     Nothing -> trumpError $ v ++ " is not defined."
eval (LetExp v e1 e2) env =
   let vval = eval e1 env
    in eval e2 ((v,vval):env)

In [20]:
p1 = IntOpExp "+" (IntOpExp "*" (IntExp 10) (IntExp 2)) (IntExp 22)
p2 = IntOpExp "+" (IntOpExp "++" (IntExp 10) (IntExp 2)) (IntExp 22)
p3 = IntOpExp "+" (IntOpExp "+" (VarExp "x") (VarExp "y")) (IntExp 22)
p5 = IntOpExp "+" (LetExp "x" (IntOpExp "*" (VarExp "x") (VarExp "y")) (VarExp "x")) (VarExp "x")
p6 = IfExp (BoolOpExp ">" (IntExp 10) (IntExp 20)) (IntExp 22) (IntExp 33)
p7 = IfExp (IntOpExp "+" (IntExp 10) (IntExp 20)) (IntExp 22) (IntExp 33)
env1 = [("x",IntVal 5), ("y",IntVal 10)]

In [21]:
eval p7 env1

You idiot! Your conditional wasn't a boolean! Sad!
CallStack (from HasCallStack):
  error, called at <interactive>:1:16 in interactive:Ghci94

In [12]:
:t lookup

lookup :: forall b a. Eq a => a -> [(a, b)] -> Maybe b

In [13]:
lookup "x" env1

Just (IntVal 5)

In [14]:
mylookup k [] = Nothing
mylookup k ((k1,v1):xs) | k == k1 = Just v1
                        | otherwise = mylookup k xs

In [15]:
x = 5
let x = 50
 in x + x

100

In [16]:
x

5